# 8.对象引用、可变性和垃圾回收
## 8.1 变量不是盒子
1. 变量不是盒子————变量不存放对象，只是对象的标记、引用
2. 对于a = 1，赋值语句，右边先执行，若右边出错，则不会创建变量

In [1]:
class Gizmo:
    def __init__(self):
        print("gizmo id:%d" %id(self))#返回对象id值
x = Gizmo()
y = Gizmo()*2


gizmo id:140300481035952
gizmo id:140300481035520


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
print(dir())#变量y未创建成功

['Gizmo', 'In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'os', 'quit', 'sys', 'x']


## 8.2 标识、相等性和别名
1. 别名指不同变量名绑定同一个对象，他们的标识（ID）也一样
2. 若不同变量绑定的对象不一样，即便对象值相同，标识也不同

In [4]:
a = [1,2,3]
b= a
c = [1,2,3]
print("别名，标识相同",a is b)
print(c is a)

别名，标识相同 True
False


3. is 运算速度比==快，== 实现对象的_\_eq__方法
3. 元组相对不可变性，元组本身不可变，元素可变（即元组的物理内容保存的引用不可变，与引用的对象无关）

In [7]:
t1 = (1,2,[30,40])
t2 = (1,2,[30,40])
print(t1==t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1[-1],id(t1[-1]))# 元组内引用元素的标识不变
print(t1==t2)

True
140421945578560
[30, 40, 99] 140421945578560
False


## 8.3 浅复制
1. list()、[:]只是对列表进行浅复制，复制初始列表的引用，只能用于初始列表中的元素都为不可变元素，否则其中一个表可变元素发生改变，其他表相应元素也会改变，互相影响
2. copy模块
    + copy.copy() 浅复制
    + copy.deepcopy()深复制 PS:注意不能有循环引用


In [13]:
a = [1,2,[3,4],(5,6)]
b = list(a)
print(id(a[2]),id(b[2]))
a[2].append(0)
print(a[2],b[2])
b[2].append(-1)
print(a[2],b[2])


print(id(a[3]),id(b[3]))
b[3] += (7,8)
print(id(a[3]),id(b[3]))
print(a[3],b[3])#元组是不可变类型，不会互相影响


140421945579072 140421945579072
[3, 4, 0] [3, 4, 0]
[3, 4, 0, -1] [3, 4, 0, -1]
140421813804928 140421813804928
140421813804928 140421014175888
(5, 6) (5, 6, 7, 8)


## 8.4 函数的参数作为引用时
1. 共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是说，函数内部的形参是实参的别名。PS：当传入可变对象时，注意函数内对形参的操作，可能影响到函数体外的实参
2. 不要使用可变类型作为参数的默认值 PS：使用可变类型作为参数的默认值，每次运行函数对该默认参数的操作结果会保留下来，影响到下一次调用函数的结果
3. 如果定义的函数接收可变参数，应该谨慎考虑调用方是否期望修改传入的参数

In [19]:
def temp(a = []):
    a.append(1)
    print(a)
temp()
temp()
temp()
print(temp.__defaults__)#默认参数把结果保留了下来

[1]
[1, 1]
[1, 1, 1]
([1, 1, 1],)


## 8.5 del和垃圾回收
1. 当对象引用归零时，对象会被销毁，释放内存
2. del不删除对象，删除对象的引用。

In [21]:
import weakref
s1 = {1,2,3}
s2 = s1
def bye():
    print("The object was destroyed!")

ender = weakref.finalize(s1,bye)#绑定弱引用
print(ender.alive)
del s1#删除s1，对象任然存活
print(ender.alive)
s2 = '1'
print(ender.alive)

True
True
The object was destroyed!
False


3. 弱引用不会增加对象的引用数量，引用的目标对象称为所指对象。不会影响对象的销毁

In [22]:
import weakref
a_set = {0,1}
wref = weakref.ref(a_set)#创建弱引用
print(wref())
a_set = {2,3}
print(wref())


{0, 1}
None
